In [33]:
#///////////////////////
# notebook des scores //////////////////////
#///////////////////////////////////////////

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import re
import unicodedata
from unidecode import unidecode
#***********************************************************************
df=pd.read_csv('Data/allocineV5-SB.csv', sep=",", index_col='ID')
df2=pd.read_csv('Data/IMDB_PRO_star.csv', sep=",")
df3=pd.read_csv('Data/Nom_imdbID_star.csv', sep=",")
df3['star']=df3['star'].replace('\n', '', regex=True)
df2['Acteur'] = df2['Acteur'].apply(lambda x : " ".join(x.split())) #on enlève les espaces avant et à la fin
df2['Acteur'] = df2['Acteur'].apply(unidecode) #on enlève les accents
df3 = df3.dropna(axis = 0, how = 'all', subset = ['star']) #' on enlève les lignes vides'
df3['star'] = df3['star'].apply(lambda x : " ".join(x.split())) #on enlève les espaces avant et à la fin
df3['star'] = df3['star'].apply(unidecode) #on enlève les accents
#*************************************************************************
#on cree un DF avec tous les acteurs uniques*******************************
#**************************************************************************
a1 = df['acteur_1']
a2 = df['acteur_2']
a3 = df['acteur_3']
a4 = df['acteur_4']
actors=pd.concat([a1,a2,a3,a4])
actors=pd.DataFrame(actors,columns=['nom'])
# 'suppression des NAN et doublons
actors = actors.drop_duplicates(keep = 'first')
actors = actors.dropna(axis = 0, how = 'any')
#suppression des accents'
actors['nom'] = actors['nom'].apply(unidecode)
#on enlève les espaces avant et à la fin
actors['nom'] = actors['nom'].apply(lambda x : " ".join(x.split())) 

#*********************************************
#******** chargement et nettoyage du allname *
#"********************************************
df4 = pd.read_csv('Data/ID_ALL_NAME.csv')
df4 = df4.dropna(axis = 0, how = 'all', subset = ['primaryName']) #' on enlève les lignes vides'
df4['primaryName'] = df4['primaryName'].apply(lambda x : " ".join(x.split())) #on enlève les espaces avant et à la fin
df4['primaryName'] = df4['primaryName'].apply(unidecode) #on enlève les accents
# df4 = df4[df4['primaryProfession'].str.contains('act')]
#*********************************************
#******** on fait les merges *****************
#*********************************************
actors = pd.merge(actors, df2, left_on = 'nom', right_on = 'Acteur', how='left').drop(columns = ['Acteur'])
# on dédoublonne
actors =actors.drop_duplicates(subset=['nom'], keep = 'first')
df3 = df3.dropna(axis = 0, how = 'all', subset = ['star']) #' on enlève les lignes vides'
''''''''''''''''''''
#on filtre actors en ne gardant que les nm vides
print(actors.shape)
actors_plein = actors.dropna(axis = 0, how = 'all', subset = ['ID'])
print(actors_plein.shape)
actors_emp= actors[actors['ID'].isna()]
actors_emp2= actors[actors['ID'].isna()]
print(actors_emp.shape,'emp')
print(actors_emp2.shape,'emp2')
#dans le all name on ne garde que les acteurs et actrices
df4 = df4[df4['primaryProfession'].isin(["actor", "actress"])]
actors_emp = pd.merge(actors_emp, df4, left_on = 'nom', right_on = 'primaryName', how='left')
actors_emp.sort_values(by=['primaryName', 'birthYear'],ascending=True)
actors_emp['primaryName'] = actors_emp['primaryName'].drop_duplicates(keep = 'first') #on enlève les doublons
actors_emp=actors_emp[['primaryName','nconst']] # on ne garde que les 2 colonnes qui nous interessent
actors_emp= actors_emp.dropna(axis = 0, how = 'all', subset = ['primaryName']) #' on enlève les lignes vides'
actors_emp3 = pd.merge(actors_emp2, actors_emp, left_on = 'nom', right_on = 'primaryName', how='left')
actors_emp3['ID']=actors_emp3['ID'].fillna(actors_emp3['nconst'])
actors_emp3=actors_emp3[['nom','ID','STARmeter']]
#******************** on enregistre
print(actors_plein.shape)
actors=pd.concat([actors_plein,actors_emp3])




actors.to_csv('Data/Actors_score.csv')
actors  

(17037, 3)
(8446, 3)
(8591, 3) emp
(8591, 3) emp2
(8446, 3)


,nom,ID,STARmeter
1,Anna Ciepielewska,nm0162134,396262.0
2,Marie Ault,nm0042083,184362.0
3,Rodolfo Arena,nm0034260,1211037.0
4,Jean Bouise,nm0099366,123746.0
5,Anne Bancroft,nm0000843,2580.0
...,...,...,...
8586,Vu Ngoc Manh,nm14765781,NaN
8587,Rezi Karosanidze,nm14849472,NaN
8588,Manon Schaap,nm10444752,NaN
8589,Ha Seong Guk,NaN,NaN
